In [2]:
import keras
from functools import reduce
import re
import numpy as np
import nltk
import json
from pprint import pprint as pp

from keras.utils.data_utils import get_file
from keras.layers.embeddings import Embedding
from keras import layers
from keras.layers import LSTM
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences

ModuleNotFoundError: No module named 'keras'

In [6]:
def tokenize(sent):
    """Returns the tokens of a sequece"""
    tokens = nltk.word_tokenize(sent)
    tokens = [w.lower() for w in tokens]
    return tokens

In [7]:
with open('data/SciQ dataset/train.json', 'r') as rf:
    train = json.load(rf)
with open('data/SciQ dataset/test.json', 'r') as rf:
    test = json.load(rf)
with open('data/SciQ dataset/valid.json', 'r') as rf:
    valid = json.load(rf)

NameError: name 'json' is not defined

In [4]:
from random import shuffle

def preprocess(data_in):
    q = []
    s = []
    o = []
    for sample in data_in:
        question = sample['question']
        support = sample['support']
        option1 = (sample['distractor1'], 0)
        option2 = (sample['distractor2'], 0)
        option3 = (sample['distractor3'], 0)
        option4 = (sample['correct_answer'], 1)
        options = [option1, option2, option3, option4]
        shuffle(options)
        print(options)
        q.append(question)
        s.append(support)
        o.append(options)
    return q, s, o

In [5]:
preprocess(train[:3])

NameError: name 'train' is not defined

In [3]:
# filename = 'data/glove.6B.50d.txt'
# UNK_ID = 2
# def loadGloVe(filename):
#     vocab_list = []
#     embd = []
#     file = open(filename,'r')
#     for line in file.readlines():
#         row = line.strip().split(' ')
#         vocab_list.append(row[0])
#         embd.append(row[1:])
#     print('Loaded GloVe!')
#     file.close()
#     vocab_size = len(vocab_list)+1
    
#     vocab = dict((c,i+1) for i, c in enumerate(vocab_list))
#     vocab['UNK_ID'] = 2
#     return vocab,vocab_list,embd
# vocab,vocab_list,embd = loadGloVe(filename)
# vocab_size = len(vocab_list)
# embedding_dim = len(embd[0])
# embedding = np.asarray(embd)

In [1]:
def createVocab(input_data):
    vocab_list = set()
    for sample in input_data:
        s_t = tokenize(sample['support'])
        q_t = tokenize(sample['question'])
        d1_t = tokenize(sample['distractor1'])
        d2_t = tokenize(sample['distractor2'])
        d3_t = tokenize(sample['distractor3'])
        a_t = tokenize(sample['correct_answer'])
        vocab_list |= set(s_t+q_t+d1_t+d2_t+d3_t+a_t)
    vocab_list=sorted(vocab_list)
    vocab_size = len(vocab_list)+3
    vocab = dict((c,i+2) for i,c in enumerate(vocab))
    return vocab_list, vocab_size, vocab

In [2]:
def get_vectors(input_sent, vocab, vocab_list):
    tokenized = tokenize(input_sent)
    vectorized = []
    for w in tokenized:
        if w in vocab_list:
            vectorized.append(vocab[w])
        else:
            vectorized.append(vocab['UNK_ID'])
    return vectorized

In [4]:
def vectorize_input(data, vocab, vocab_size, support_maxlen, query_maxlen):
    qs = []
    sps = []
    ans = []
    dis1s = []
    dis2s = []
    dis3s = []
    for _,sample in enumerate(data):
        q_vect = get_vectors(sample['question'], vocab, vocab_list)
        support_vect = get_vectors(sample['support'], vocab , vocab_list)
        ans_tokens = tokenize(sample['correct_answer'])
        answer_vect = np.zeros(vocab_size)
        for w in ans_tokens:
            if w in vocab_list:
                answer_vect[vocab[w]]=1
            else:
                answer_vect[vocab['UNK_ID']]=1
                
        answer_vect = np.transpose(answer_vect)
                    
                                
        distractor1_vect = get_vectors(sample['distractor1'], vocab , vocab_list)
        distractor2_vect = get_vectors(sample['distractor2'], vocab , vocab_list)
        distractor3_vect = get_vectors(sample['distractor3'], vocab , vocab_list)
        qs.append(q_vect)
        sps.append(support_vect)
        ans.append(answer_vect)
        dis1s.append(distractor1_vect)
        dis2s.append(distractor2_vect)
        dis3s.append(distractor3_vect)
    return(pad_sequences(qs, maxlen=query_maxlen),\
           pad_sequences(sps, maxlen=support_maxlen),\
           np.array(ans),\
           pad_sequences(dis1s, maxlen=query_maxlen),\
           pad_sequences(dis2s, maxlen=query_maxlen),\
           pad_sequences(dis3s, maxlen=query_maxlen)
          )

In [5]:

#vectorize_input(train[:5], vocab, vocab_list, 500, 100)

In [6]:
EMBED_SIZE = 300
Q_HIDDEN_SIZE = 100
S_HIDDEN_SIZE = 300
BATCH_SIZE = 32
EPOCHS = 40
print('LSTM/EMBED/SUPPORT/QUERY={0},{1},{2},{3}'.format(LSTM,
                                                    EMBED_SIZE,
                                                    S_HIDDEN_SIZE,
                                                    Q_HIDDEN_SIZE))


LSTM/EMBED/SUPPORT/QUERY=<class 'keras.layers.recurrent.LSTM'>,300,300,100


In [ ]:
q,s,a,d1,d2,d3 = vectorize_input(train, vocab, vocab_size, S_HIDDEN_SIZE, Q_HIDDEN_SIZE)
support = layers.Input(shape=(S_HIDDEN_SIZE,), dtype='int32')
encoded_support = layers.Embedding(vocab_size, EMBED_SIZE)(support)
encoded_support = layers.Dropout(0.3)(encoded_support)
support_LSTM = LSTM(EMBED_SIZE)(encoded_support)
support_LSTM = layers.RepeatVector(S_HIDDEN_SIZE)(support_LSTM)

In [ ]:
question = layers.Input(shape=(Q_HIDDEN_SIZE,), dtype='int32')
encoded_question = layers.Embedding(vocab_size, EMBED_SIZE)(question)
encoded_question = layers.Dropout(0.3)(encoded_question)
question_LSTM = LSTM(EMBED_SIZE)(encoded_question)
question_LSTM = layers.RepeatVector(S_HIDDEN_SIZE)(question_LSTM)

In [ ]:
merged = layers.add([support_LSTM, question_LSTM])
match_LSTM = LSTM(EMBED_SIZE)(merged)
match_LSTM = layers.Dropout(0.3)(match_LSTM)
predictions = layers.Dense(vocab_size, activation='softmax')(match_LSTM)
print(predictions)

In [ ]:
model = Model([support, question], predictions)
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
print('Training')
model.fit([s,q], a,
         batch_size=BATCH_SIZE,
         epochs=EPOCHS)

Training


/home/prajnya/anaconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:97: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 120000000 elements. This may consume a large amount of memory.
  num_elements)


Epoch 1/40
